In [ ]:
import os
import shutil
import pickle
import random
import sys
import numpy as np
import pandas as pd
import duckdb
from typing import List

import requests
from io import StringIO

import ibis
import ibis.selectors as s
from ibis import _
ibis.options.interactive = True

from darts import TimeSeries, concatenate
from darts.dataprocessing.transformers import (
    Scaler,
    MissingValuesFiller,
    Mapper,
    InvertibleMapper,
)
from darts.dataprocessing import Pipeline
from darts.metrics import mape, smape, mae, ope, rmse
from darts.utils.statistics import check_seasonality, plot_acf
from darts.datasets import AirPassengersDataset, IceCreamHeaterDataset
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.utils.likelihood_models import QuantileRegression, GumbelLikelihood, GaussianLikelihood

from darts import TimeSeries
from darts.utils.timeseries_generation import (
    gaussian_timeseries,
    linear_timeseries,
    sine_timeseries,
)
from darts.models import (
    TFTModel,
    TiDEModel,
    DLinearModel,
    NLinearModel,
    TSMixerModel
)


from torchmetrics import (
    SymmetricMeanAbsolutePercentageError, 
    MeanAbsoluteError, 
    MeanSquaredError,
)

from pytorch_lightning.callbacks.early_stopping import EarlyStopping

import warnings
warnings.filterwarnings("ignore")

# logging
import logging

# define log
logging.basicConfig(level=logging.INFO)
log = logging.getLogger(__name__)

In [ ]:
import optuna
from optuna.integration import PyTorchLightningPruningCallback
from optuna.visualization import (
    plot_optimization_history,
    plot_contour,
    plot_param_importances,
    plot_pareto_front,
)

In [ ]:
## will be loaded from root when deployed
from darts_wrapper import DartsGlobalModel

In [ ]:
os.chdir('../..')

In [ ]:
# custom modules
import src.data_engineering as de
from src import plotting
from src import utils

## Data prep

In [ ]:
# connect to database
con = ibis.duckdb.connect("data/spp.ddb")
con.list_tables()

In [ ]:
lmp = con.table('lmp')
lmp

In [ ]:
lmp = lmp.filter(_.Settlement_Location_Name.contains('PSCO'))

In [ ]:
lmp.to_pandas()[['GMTIntervalEnd_HE', 'Settlement_Location_Name']].duplicated().sum()

In [ ]:
drop_cols = [
    'Interval_HE', 'GMTIntervalEnd_HE', 'timestamp_mst_HE',
    'Settlement_Location_Name', 'PNODE_Name', 
    'MLC', 'MCC', 'MEC'
]

lmp = (
    lmp
    .mutate(unique_id = _.Settlement_Location_Name )
    .mutate(timestamp_mst = _.timestamp_mst_HE)
    # .mutate(y = _.LMP) 
    .drop(drop_cols) 
    .order_by(['unique_id', 'timestamp_mst'])
)

lmp

In [ ]:
mtrf = con.table('mtrf')
mtrf

In [ ]:
drop_cols = ['Interval', 'GMTIntervalEnd']

mtrf = (
    mtrf
    # .mutate(ds = _.timestamp_mst)
    .drop(drop_cols) 
    .order_by(['timestamp_mst'])
)

mtrf

In [ ]:
mtlf = con.table('mtlf')
mtlf

In [ ]:
drop_cols = ['Interval', 'GMTIntervalEnd',]

mtlf = (
    mtlf
    # .mutate(ds = _.timestamp_mst)
    .drop(drop_cols) 
    .order_by(['timestamp_mst'])
)

mtlf

In [ ]:
all_df = (
    mtlf
    .left_join(mtrf, 'timestamp_mst')
    .select(~s.contains("_right")) # remove 'dt_right'
    .left_join(lmp, 'timestamp_mst')
    .select(~s.contains("_right")) # remove 'dt_right'
    .order_by(['unique_id', 'timestamp_mst'])
)
all_df 


In [ ]:
all_df.count()

In [ ]:
all_df = (
    all_df
    .drop_null(['unique_id'])
    # .mutate(hour = _.ds.hour())
    # .mutate(weekday = _.ds.day_of_week.index())
    # .mutate(month = _.ds.month())
    .mutate(re_ratio = (_.Wind_Forecast_MW + _.Solar_Forecast_MW) / _.MTLF)
)

all_df

In [ ]:
all_df_pd = all_df.to_pandas()
all_df_pd

In [ ]:
all_df_pd.info()

In [ ]:
len(all_df_pd.timestamp_mst.unique()) * len(all_df_pd.unique_id.unique())

In [ ]:
node_groups = all_df_pd.unique_id.unique()
log.info(f'number of nodes: {len(node_groups)}')
node_groups

In [ ]:
node_groups = [node for node in node_groups if 'PSCO_' in node]
log.info(f'number of nodes: {len(node_groups)}')
node_groups

In [ ]:
all_df_pd = all_df_pd[all_df_pd.unique_id.isin(node_groups)].reset_index(drop=True)
all_df_pd

In [ ]:
# all_df_pd.drop(['Averaged_Actual'], axis=1, errors='ignore', inplace=True)
all_df_pd.set_index('timestamp_mst', inplace=True)
all_df_pd

## Prep model training

In [ ]:
FORECAST_HORIZON = 24*5
INPUT_CHUNK_LENGTH = 2*FORECAST_HORIZON

In [ ]:
futr_cols = ['MTLF', 'Wind_Forecast_MW', 'Solar_Forecast_MW', 're_ratio']
past_cols = ['Averaged_Actual']
y = ['LMP']
ids = ['unique_id']

In [ ]:
all_df_pd = all_df_pd[ids + y + past_cols + futr_cols]
all_df_pd 

In [ ]:
train_start = all_df_pd.index.min() + pd.Timedelta(f'{2*INPUT_CHUNK_LENGTH}h')
test_end = all_df_pd.index.max() - pd.Timedelta(f'{2*FORECAST_HORIZON}h')
tr_tst_split =  test_end - pd.Timedelta(f'{2*INPUT_CHUNK_LENGTH}h')
log.info(f'train_start: {train_start}')
log.info(f'tr_tst_split: {tr_tst_split}')
log.info(f'test_end: {test_end}')

In [ ]:
train_idx = (all_df_pd.index < tr_tst_split) & (all_df_pd.index > train_start)
test_idx = (all_df_pd.index > tr_tst_split) & (all_df_pd.index < test_end)
train_all = all_df_pd[train_idx]
train_all

In [ ]:
test_all = all_df_pd[test_idx]
test_all

In [ ]:
def fill_missing(series):
    for i in range(len(series)):
        transformer = MissingValuesFiller()
        series[i] = transformer.transform(series[i])

In [ ]:
all_series = TimeSeries.from_group_dataframe(
    all_df_pd,
    group_cols=ids,
    value_cols=y,
    fill_missing_dates=True,
    freq='h',
)

fill_missing(all_series) 
all_series[0].plot()

In [ ]:
train_series = TimeSeries.from_group_dataframe(
    train_all,
    group_cols=ids,
    value_cols=y,
    fill_missing_dates=True,
    freq='h',
)
fill_missing(train_series)
train_series[0].plot()

In [ ]:
test_series = TimeSeries.from_group_dataframe(
    test_all,
    group_cols=ids,
    value_cols=y,
    fill_missing_dates=True,
    freq='h',
)
fill_missing(test_series)
test_series[0].plot()

In [ ]:
futr_cov = TimeSeries.from_group_dataframe(
    all_df_pd,
    group_cols=ids,
    value_cols=futr_cols,
    fill_missing_dates=True,
    freq='h',
)
fill_missing(futr_cov)
futr_cov[0].plot()

In [ ]:
past_cov = TimeSeries.from_group_dataframe(
    all_df_pd,
    group_cols=ids,
    value_cols=past_cols,
    fill_missing_dates=True,
    freq='h',
)
fill_missing(past_cov)
past_cov[0].plot()

In [ ]:
# test_futr_cov = TimeSeries.from_group_dataframe(
#     test_all,
#     group_cols=ids,
#     value_cols=cov_cols,
#     fill_missing_dates=True,
#     freq='h',
# )

# test_futr_cov[0].plot()

## MLFlow setup

In [ ]:
import mlflow

In [ ]:
# mlflow.set_tracking_uri("sqlite:///mlruns.db")
mlflow.get_tracking_uri()

In [ ]:
exp_name = 'spp_weis'

if mlflow.get_experiment_by_name(exp_name) is None:
    exp = mlflow.create_experiment(exp_name)
    
exp = mlflow.get_experiment_by_name(exp_name)
exp

## Get model signature

In [ ]:
node_series = train_series[0]
future_cov_series = futr_cov[0]
past_cov_series = past_cov[0]

data = {
    'series': [node_series.to_json()],
    'past_covariates': [past_cov_series.to_json()],
    'future_covariates': [future_cov_series.to_json()],
    'n': FORECAST_HORIZON,
    'num_samples': 200
}
df = pd.DataFrame(data)

ouput_example = 'the endpoint return json as a string'

In [ ]:
from mlflow.models import infer_signature
darts_signature = infer_signature(df, ouput_example)
darts_signature

## Set up hyperparameter tuning study

https://unit8co.github.io/darts/examples/17-hyperparameter-optimization.html?highlight=optuna

In [ ]:
import torch
import pprint
# set up pretty printer
pp = pprint.PrettyPrinter(indent=2, sort_dicts=False)

def log_pretty(obj):
    pretty_out = f"{pp.pformat(obj)}"

    return f'{pretty_out}\n'
    
def build_fit_tsmixerx(
    series: List[TimeSeries]=train_series,
    val_series: List[TimeSeries]=test_series,
    future_covariates: List[TimeSeries]=futr_cov,
    past_covariates: List[TimeSeries]=past_cov,
    hidden_size: int=32,
    ff_size: int=128,
    num_blocks: int=2,
    forecast_horizon: int=FORECAST_HORIZON,
    input_chunk_length: int=INPUT_CHUNK_LENGTH,
    lr: float=1e-4,
    batch_size: int=64,
    n_epochs: int=4,
    force_reset: bool=True, # reset model if already exists
    callbacks=None,
):
    work_dir = os.getcwd() + '/model_checkpoints'
    MODEL_TYPE = "ts_mixer_model"
    quantiles = [0.01]+np.arange(0.05, 1, 0.05).tolist()+[0.99]
    
    #TODO: pick a metric...
    torch_metrics = MeanAbsoluteError()
    # torch_metrics = MeanSquaredError(squared=False)
    # torch_metrics = SymmetricMeanAbsolutePercentageError()
    
    encoders = {
        "datetime_attribute": {
            "future": ["hour", "dayofweek", "month"], # 
            "past": ["hour", "dayofweek", "month"], # 
        },
        "position": {
            "past": ["relative"], 
            "future": ["relative"]
        },
        "transformer": Scaler(global_fit=True)
    }

    # common parameters across models
    model_params = {
        'hidden_size': hidden_size,
        'ff_size': ff_size,
        'num_blocks': num_blocks,
        'input_chunk_length': input_chunk_length,
        'output_chunk_length': forecast_horizon,
        'batch_size': batch_size,
        'n_epochs': n_epochs,
        'add_encoders': encoders,
        'likelihood': QuantileRegression(quantiles=quantiles),  # QuantileRegression is set per default
        'optimizer_kwargs': {"lr": lr},
        'random_state': 42,
        'torch_metrics': torch_metrics,
        'use_static_covariates': False,
        'save_checkpoints': True,
        'work_dir': work_dir,
        'model_name': MODEL_TYPE, # used for checkpoint saves
        'force_reset': force_reset, # reset model if already exists
    }
    log.info(f'model_params: \n{log_pretty(model_params)}')

    # throughout training we'll monitor the validation loss for early stopping
    early_stopper = EarlyStopping("val_loss", min_delta=0.01, patience=3, verbose=True)
    if callbacks is None:
        callbacks = [early_stopper]
    else:
        callbacks = [early_stopper] + callbacks

    # detect if a GPU is available
    if torch.cuda.is_available():
        pl_trainer_kwargs = {
            "accelerator": "gpu",
            "gpus": -1,
            "auto_select_gpus": True,
            "callbacks": callbacks,
        }
    else:
        pl_trainer_kwargs = {"callbacks": callbacks}
        

    model = TSMixerModel(**model_params)

    # train the model
    fit_params = {
        'series': train_series,
        'val_series': test_series,
        'future_covariates': futr_cov,
        'past_covariates': past_cov,
        'val_future_covariates': futr_cov,
        'val_past_covariates': past_cov,
    }
    model.fit(**fit_params)

    # reload best model over course of training
    model = TSMixerModel.load_from_checkpoint(
        work_dir=work_dir,
        model_name=MODEL_TYPE
    )
    
    model.MODEL_TYPE = MODEL_TYPE

    return model
    

In [ ]:
def build_fit_tft(
    series: List[TimeSeries]=train_series,
    val_series: List[TimeSeries]=test_series,
    future_covariates: List[TimeSeries]=futr_cov,
    past_covariates: List[TimeSeries]=past_cov,
    hidden_size: int=16, # Hidden state size of the TFT. It is the main hyper-parameter and common across the internal TFT architecture.
    lstm_layers: int = 1, # Number of layers for the Long Short Term Memory (LSTM) Encoder and Decoder (1 is a good default).
    num_attention_heads: int=2, # Number of attention heads (4 is a good default)
    dropout: float=0.1,
    forecast_horizon: int=FORECAST_HORIZON,
    input_chunk_length: int=INPUT_CHUNK_LENGTH,
    lr: float=1e-4,
    batch_size: int=64,
    n_epochs: int=4,
    force_reset: bool=True, # reset model if already exists
    callbacks=None,
):
    work_dir = os.getcwd() + '/model_checkpoints'
    MODEL_TYPE = "tft_model"
    quantiles = [0.01]+np.arange(0.05, 1, 0.05).tolist()+[0.99]
    
    #TODO: pick a metric...
    torch_metrics = MeanAbsoluteError()
    # torch_metrics = MeanSquaredError(squared=False)
    # torch_metrics = SymmetricMeanAbsolutePercentageError()
    
    encoders = {
        "datetime_attribute": {
            "future": ["hour", "dayofweek", "month"], # 
            "past": ["hour", "dayofweek", "month"], # 
        },
        "position": {
            "past": ["relative"], 
            "future": ["relative"]
        },
        "transformer": Scaler(global_fit=True)
    }

    # common parameters across models
    model_params = {
        'hidden_size': hidden_size,
        'lstm_layers': lstm_layers,
        'num_attention_heads': num_attention_heads,
        'dropout': dropout,
        'input_chunk_length': input_chunk_length,
        'output_chunk_length': forecast_horizon,
        'batch_size': batch_size,
        'n_epochs': n_epochs,
        'add_encoders': encoders,
        'likelihood': QuantileRegression(quantiles=quantiles),  # QuantileRegression is set per default
        'optimizer_kwargs': {"lr": lr},
        'random_state': 42,
        'torch_metrics': torch_metrics,
        'use_static_covariates': False,
        'save_checkpoints': True,
        'work_dir': work_dir,
        'model_name': MODEL_TYPE, # used for checkpoint saves
        'force_reset': force_reset, # reset model if already exists
    }
    log.info(f'model_params: \n{log_pretty(model_params)}')

    # throughout training we'll monitor the validation loss for early stopping
    early_stopper = EarlyStopping("val_loss", min_delta=0.01, patience=3, verbose=True)
    if callbacks is None:
        callbacks = [early_stopper]
    else:
        callbacks = [early_stopper] + callbacks

    # detect if a GPU is available
    if torch.cuda.is_available():
        pl_trainer_kwargs = {
            "accelerator": "gpu",
            "gpus": -1,
            "auto_select_gpus": True,
            "callbacks": callbacks,
        }
    else:
        pl_trainer_kwargs = {"callbacks": callbacks}
        

    model = TFTModel(**model_params)

    # train the model
    fit_params = {
        'series': train_series,
        'val_series': test_series,
        'future_covariates': futr_cov,
        'past_covariates': past_cov,
        'val_future_covariates': futr_cov,
        'val_past_covariates': past_cov,
    }
    model.fit(**fit_params)

    # reload best model over course of training
    model = TFTModel.load_from_checkpoint(
        work_dir=work_dir,
        model_name=MODEL_TYPE
    )
    
    model.MODEL_TYPE = MODEL_TYPE

    return model

In [ ]:
# test build fit function
# model = build_fit_tsmixerx(n_epochs=1)
model = build_fit_tft(n_epochs=1)

In [ ]:
model.MODEL_TYPE

In [ ]:
model.model_params

In [ ]:
preds = model.predict(
        series=train_series, 
        n=FORECAST_HORIZON,
        past_covariates=past_cov,
        future_covariates=futr_cov,
        num_samples=200,
)

smapes = smape(test_series, preds, n_jobs=-1, verbose=True)
smape_val = np.mean(smapes)
smape_val

In [ ]:
### retest this....
# acc = model.backtest(
#     series=test_series,
#     past_covariates=past_cov,
#     future_covariates=futr_cov,
#     retrain=False,
#     forecast_horizon=FORECAST_HORIZON,
#     stride=25,
#     metric=[get_ci_error],
#     verbose=False,
#     num_samples=200,
    # retrain=False,
# )

In [ ]:
from darts.metrics.metrics import multi_ts_support, multivariate_support

# @multivariate_support
def get_ci_error(actual_series, pred_series, n_jobs=1, verbose=False):
    series_qs = pred_series.quantiles_df((0.2, 0.8))
    val_y = actual_series.pd_dataframe()
    
    eval_df = series_qs.merge(
        val_y,
        how='inner',
        left_index=True,
        right_index=True,
    )

    # 0 if u > l
    upper_error = (
        (eval_df['LMP_0.8'] - eval_df['LMP']) # upper - pred
        .apply(lambda x: min(x, 0)) # min(u-p, 0)
        .abs().mean() # sum(abs(min(u-p, 0)))
    )

    # 0 if p > l
    lower_error = (
        (eval_df['LMP_0.2'] - eval_df['LMP']) # lower - pred
        .apply(lambda x: max(x, 0)) # max(l-p, 0)
        .abs().mean() # sum(abs(max(l-p, 0)))
    )

    return upper_error + lower_error

In [ ]:
def get_ci_cover_err(actual_series, pred_series, n_jobs=1, verbose=False):
    series_qs = pred_series.quantiles_df((0.1, 0.9))
    val_y = actual_series.pd_dataframe()
    
    eval_df = series_qs.merge(
        val_y,
        how='inner',
        left_index=True,
        right_index=True,
    )

    # 0 if u > l
    cover = (
        (eval_df['LMP_0.9'] < eval_df['LMP']) |
        (eval_df['LMP_0.1'] > eval_df['LMP'])
    ).mean() # should be about 80%

    return 100 * np.abs(cover - 0.8)

In [ ]:
def objective_tsmixer(trial):
    callback = [PyTorchLightningPruningCallback(trial, monitor="val_loss")]

    # Hyperparameters
    hidden_size = trial.suggest_int("hidden_size", 16, 64)
    ff_size = trial.suggest_int("ff_size", 64, 256)
    num_blocks = trial.suggest_int("num_blocks", 1, 4)
    lr = trial.suggest_float("lr", 1e-4, 5e-4, step=1e-4)
    n_epochs = trial.suggest_int("n_epochs", 3, 10)
    

    # build and train the TCN model with these hyper-parameters:
    model = build_fit_tsmixerx(
        hidden_size=hidden_size,
        ff_size=ff_size,
        num_blocks=num_blocks,
        lr=lr,
        n_epochs=n_epochs,
        callbacks=callback,
    )

    # Evaluate how good it is on the validation set
    preds = model.predict(
        series=train_series, 
        n=len(test_series[0]),
        past_covariates=past_cov,
        future_covariates=futr_cov,
        num_samples=200,
    )

    smapes = smape(test_series, preds, n_jobs=-1, verbose=True)
    smape_val = np.mean(smapes)
    
    if smape_val!= np.nan:
        pass
    else:
        smape_val = float("inf")

    ci_error = np.mean([get_ci_cover_err(test_series[i], preds[i]) for i in range(len(preds))])
    

    return smape_val , ci_error



In [ ]:
def objective_tft(trial):
    callback = [PyTorchLightningPruningCallback(trial, monitor="val_loss")]

    # Hyperparameters
    hidden_size = trial.suggest_int("hidden_size", 8, 32)
    lstm_layers = trial.suggest_int("lstm_layers", 1, 2)
    num_attention_heads = trial.suggest_int("num_attention_heads", 1, 4)
    lr = trial.suggest_float("lr", 1e-4, 5e-4, step=1e-4)
    n_epochs = trial.suggest_int("n_epochs", 3, 6)
    

    # build and train the TCN model with these hyper-parameters:
    model = build_fit_tft(
        hidden_size=hidden_size,
        lstm_layers=lstm_layers,
        num_attention_heads=num_attention_heads,
        lr=lr,
        n_epochs=n_epochs,
        callbacks=callback,
    )

    # Evaluate how good it is on the validation set
    preds = model.predict(
        series=train_series, 
        n=len(test_series[0]),
        past_covariates=past_cov,
        future_covariates=futr_cov,
        num_samples=200,
    )

    smapes = smape(test_series, preds, n_jobs=-1, verbose=True)
    smape_val = np.mean(smapes)
    
    if smape_val!= np.nan:
        pass
    else:
        smape_val = float("inf")

    ci_error = np.mean([get_ci_cover_err(test_series[i], preds[i]) for i in range(len(preds))])
    

    return smape_val , ci_error

In [ ]:
def print_callback(study, trial):
    best_smape = min(study.best_trials, key=lambda t: t.values[0])
    best_ci = min(study.best_trials, key=lambda t: t.values[1])
    log.info(f"Current values: {trial.values}, Current params: \n{log_pretty(trial.params)}")
    log.info(f"Best SMAPE: {best_smape.values}, Best params: \n{log_pretty(best_smape.params)}")
    log.info(f"Best CI: {best_ci.values}, Best params: \n{log_pretty(best_ci.params)}")

In [ ]:
study = optuna.create_study(
    directions=["minimize", "minimize"],
    storage="sqlite:///spp_trials.db", 
    study_name="spp_weis_tft",
    load_if_exists=True,
)

study.optimize(objective_tft, n_trials=10, callbacks=[print_callback])

In [ ]:
# study = optuna.create_study(
#     directions=["minimize", "minimize"],
#     storage="sqlite:///spp_trials.db", 
#     study_name="spp_weis",
#     load_if_exists=True,
# )

# study.optimize(objective_tsmixer, n_trials=40, callbacks=[print_callback])
# # or limit time
# # study.optimize(objective, timeout=7200, callbacks=[print_callback])

In [ ]:
plot_optimization_history(study, target=lambda t: t.values[0])

In [ ]:
plot_optimization_history(study, target=lambda t: t.values[1])

In [ ]:
plot_contour(study, params=["lr", "ff_size"], target=lambda t: t.values[0])

In [ ]:
plot_param_importances(study)

In [ ]:
plot_pareto_front(study, target_names=["SMAPE", "CI Error"])

In [ ]:
# TODO: think about how to wieght values to get best model
best_model = min(study.best_trials, key=lambda t: sum(t.values))
log.info(f"Best number: {best_model.number}")
log.info(f"Best values: {best_model.values}")
log.info(f"Best params: \n{log_pretty(best_model.params)}")

## Refit and log model with best params

In [ ]:
with mlflow.start_run(experiment_id=exp.experiment_id) as run:
    # fit model with best params from study
    model = build_fit_tsmixerx(**best_model.params)
    
    log.info(f'run.info: \n{run.info}')
    artifact_path = "model_artifacts"
    metrics = {}
    params = model.model_params
    
    # back test on validation data
    acc = model.backtest(
        series=test_series,
        # series=all_series,
        past_covariates=past_cov,
        future_covariates=futr_cov,
        retrain=False,
        forecast_horizon=params['output_chunk_length'],
        stride=25,
        metric=[mae, rmse],
        verbose=False,
    )

    log.info(f'BACKTEST: acc: {acc}')
    log.info(f'BACKTEST: np.mean(acc, axis=0): {np.mean(acc, axis=0)}')
    acc_df = pd.DataFrame(
        np.mean(acc, axis=0).reshape(1,-1),
        columns=['mae', 'rmse']
    )

    # add metrics
    metrics['test_mae'] = acc_df.mae[0]
    metrics['test_rmse'] = acc_df.rmse[0]

    # final training
    final_train_series = test_series
    log.info('final training')
    model.fit(
            series=test_series,
            past_covariates=past_cov,
            future_covariates=futr_cov,
            verbose=True,
            # epochs=params['n_epochs_final'], # continue training
            )
    
    # final model back test on validation data
    acc = model.backtest(
            series=test_series,
            past_covariates=past_cov,
            future_covariates=futr_cov,
            retrain=False,
            forecast_horizon=params['output_chunk_length'],
            stride=25,
            metric=[mae, rmse],
            verbose=False,
        )

    log.info(f'TEST ACC: acc: {acc}')
    log.info(f'TEST ACC: np.mean(acc, axis=0): {np.mean(acc, axis=0)}')
    acc_df = pd.DataFrame(
        np.mean(acc, axis=0).reshape(1,-1),
        columns=['mae', 'rmse']
    )

    # add and log metrics
    metrics['mae_final'] = acc_df.mae[0]
    metrics['rmse_final'] = acc_df.rmse[0]
    mlflow.log_metrics(metrics)

    # set up path to save model
    model_path = '/'.join([artifact_path, model.MODEL_TYPE])

    shutil.rmtree(artifact_path, ignore_errors=True)
    os.makedirs(artifact_path)

    # log params
    mlflow.log_params(params)

    # save model files (model, model.ckpt) 
    # and load them to artifacts when logging the model
    model.save(model_path)

    # save MODEL_TYPE to artifacts
    # this will be used to load the model from the artifacts
    model_type_path = '/'.join([artifact_path, 'MODEL_TYPE.pkl'])
    with open(model_type_path, 'wb') as handle:
        pickle.dump(model.MODEL_TYPE, handle)
    
    # map model artififacts in dictionary
    artifacts = {
        'model': model_path,
        'model.ckpt': model_path+'.ckpt',
        'MODEL_TYPE': model_type_path,
    }
    
    # log model
    # https://www.mlflow.org/docs/latest/tutorials-and-examples/tutorial.html#pip-requirements-example
    mlflow.pyfunc.log_model(
        artifact_path='GlobalForecasting',
        code_path=['notebooks/model_training/darts_wrapper.py'],
        signature=darts_signature,
        artifacts=artifacts,
        # model will get loaded from artifacts, we don't need instantiate with one
        python_model=DartsGlobalModel(), 
        pip_requirements=["-r notebooks/model_training/requirements.txt"],
    )


## Get latest run and test predicting

In [ ]:
runs = mlflow.search_runs(
    experiment_ids = exp.experiment_id,
    # order_by=['metrics.test_mae']
    order_by=['end_time']
    )

runs.sort_values('end_time', ascending=False, inplace=True)
runs

In [ ]:
best_run_id = runs.run_id.iloc[0]
best_run_id

In [ ]:
runs['artifact_uri'].iloc[0]

In [ ]:
model_path = runs['artifact_uri'].iloc[0] + '/GlobalForecasting'

In [ ]:
loaded_model = mlflow.pyfunc.load_model(model_path)

In [ ]:
plot_series = all_series[0]
# plot_end_time = min(plot_series.end_time(), pd.Timestamp('2024-07-01T23:00:00'))
# plot_end_time = min(plot_series.end_time(), pd.Timestamp('2024-07-15T23:00:00'))
plot_end_time = min(plot_series.end_time(), pd.Timestamp('2024-07-14T23:00:00'))
plot_end_time = min(plot_series.end_time(), pd.Timestamp('2024-07-20T23:00:00'))
plot_end_time = min(plot_series.end_time(), pd.Timestamp('2024-08-01T23:00:00'))
log.info(F'plot_end_time: {plot_end_time}')

In [ ]:
plot_node_name = plot_series.static_covariates.unique_id.LMP
plot_node_name

In [ ]:
# if test_end_time < test_series.end_time():
node_series = plot_series.drop_after(plot_end_time)
    
log.info(f'plot_end_time: {plot_end_time}')
log.info(f'node_series.end_time(): {node_series.end_time()}')
future_cov_series = futr_cov[0]
past_cov_series = past_cov[0]


data = {
    'series': [node_series.to_json()],
    'past_covariates': [past_cov_series.to_json()],
    'future_covariates': [future_cov_series.to_json()],
    'n': FORECAST_HORIZON,
    'num_samples': 200
}
df = pd.DataFrame(data)

plot_cov_df = future_cov_series.pd_dataframe()
plot_cov_df = (
    plot_cov_df
    .reset_index()
    .rename(columns={'timestamp_mst':'time', 're_ratio': 'Ratio'})
)
plot_cov_df

In [ ]:
df

In [ ]:
# Predict on a Pandas DataFrame.
df['num_samples'] = 500
pred = loaded_model.predict(df)
preds = TimeSeries.from_json(pred)

In [ ]:
q_df = plotting.get_quantile_df(preds)
q_df

In [ ]:
plot_df = plotting.get_mean_df(preds).merge(
    plotting.get_quantile_df(preds),
    left_index=True,
    right_index=True,
)
plot_df 

In [ ]:
lmp_df = lmp.to_pandas().rename(
    columns={
        'LMP': 'LMP_HOURLY',
        'unique_id':'node', 
        'timestamp_mst':'time'
    })

In [ ]:
plot_df = plotting.get_plot_df(
        TimeSeries.from_json(pred),
        plot_cov_df,
        lmp_df,
        plot_node_name,
    )
plot_df.rename(columns={'mean':'mean_fcast'}, inplace=True)
plot_df

plotting.plotly_forecast(plot_df, plot_node_name)